In [5]:
import numpy
import random

import numpy as np
import pandas as pd

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

from scoop import futures
import pickle

stock = "TSLA"
trends_data = np.loadtxt(f"Datasets/{stock}/{stock}validation.csv",delimiter=',', skiprows=1) 


In [6]:
def evaluation(individual):
    fitness = 0 
    for i in range(len(trends_data)-1):
        individual_arr = np.array(individual)
        trend = trends_data[i,1:].dot(individual_arr)
        if trend > 3:
            if trends_data[i+1, 0] > trends_data[i, 0]: 
                fitness +=1 
        elif trend <= 0:
            if trends_data[i+1, 0] < trends_data[i, 0]: 
                fitness +=1 
        else:
            if abs(trends_data[i+1, 0] == trends_data[i, 0]) < 1e-5:
                fitness +=1 
    return fitness,


In [7]:
ind = [0, 1, 0, 0, 1, 1, -1, 1, 0] 
evaluation(ind)

(770,)

In [8]:
len(trends_data)

772

In [4]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)



In [5]:
def evalOneMax(individual):
    return sum(individual),

In [6]:
toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("weights", random.randint, 1, 3)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual,
    toolbox.weights, 9)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# Evaluation function
toolbox.register("evaluate", evaluation)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, indpb=0.05, low=1, up=3)
toolbox.register("select", tools.selTournament, tournsize=10)
toolbox.register("map", futures.map)

In [11]:
ind = [1, 1, 3, 1, 2, 3, 1, 2, 1]
evaluation(ind)

(882,)

In [9]:
def main(checkpoint=None):
    pkl = "GA4.pkl"
    n = 256
    if checkpoint:
        # A file name has been given, then load the data from the file
        with open(checkpoint, "rb") as cp_file:
            cp = pickle.load(cp_file)
        pop = cp["pop"]
        start_gen = cp["generation"]
        halloffame = cp["halloffame"]
        logbook = cp["logbook"]
        random.setstate(cp["rndstate"])
    else:
        # Start a new evolution
        pop = toolbox.population(n=n)
        start_gen = 0
        halloffame = tools.HallOfFame(maxsize=1)
        logbook = tools.Logbook()

    # Evaluate the entire population
    fitnesses = list(toolbox.map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    CXPB, MUTPB = 0.5, 0.1
    # Extracting all the fitnesses of the population
    fits = [ind.fitness.values[0] for ind in pop]
    # Variable keeping track of the number of generations
    gen = start_gen

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("max", np.max)
    stats.register("min", np.min)
    stats.register("std", np.std)
    while gen <= 2000:
        # A new generation
        gen = gen + 1
        # Select the next generation individuals
        offspring = toolbox.select(pop, n)
        # Clone the selected individuals
        offspring = list(toolbox.map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        pop[:] = offspring

        halloffame.update(pop)
        record = stats.compile(pop)
        logbook.record(gen=gen, evals=len(invalid_ind), **record)
        if gen % 20 == 0:
            # Fill the dictionary using the dict(key=value[, ...]) constructor
            cp = dict(pop=pop, generation=gen, halloffame=halloffame,
                      logbook=logbook, rndstate=random.getstate())

            with open(pkl, "wb") as cp_file:
                pickle.dump(cp, cp_file)


        print(logbook.stream)


In [10]:
main()

/home/vincent/miniconda3/envs/stockprices/lib/python3.8/site-packages/scoop/fallbacks.py:38: RuntimeWarning: SCOOP was not started properly.
Be sure to start your program with the '-m scoop' parameter. You can find further information in the documentation.
Your map call has been replaced by the builtin serial Python map().
  warnings.warn(


avg    	evals	gen	max	min	std    
784.543	124  	1  	874	624	54.4562
817.047	136  	2  	874	619	59.1188
846.957	131  	3  	874	699	44.8262
857.715	134  	4  	874	699	43.2544
858.477	148  	5  	874	627	46.3523
858.797	141  	6  	874	627	45.5943
868.371	128  	7  	874	696	26.814 


KeyboardInterrupt: 